BASIC OBJECT TRACKING TECHNIQUES:

1) **OPTICAL FLOW**

2) **MeanShift AND CamShift**

**Optical flow** is the pattren of apparent motion of image objects between two consecutive frames caused by the movement of object or camera.

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [3]:
lk_params = dict(winSize = (200,200),maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [4]:
cap = cv2.VideoCapture(0)

ret, previous_frame = cap.read()

previous_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

# POINTS TO TRACK
previousPoints = cv2.goodFeaturesToTrack(previous_gray, mask = None, **corner_track_params)

mask = np.zeros_like(previous_frame)

while True:
    
    ret, frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    nextPoints, status, err = cv2.calcOpticalFlowPyrLK(previous_gray, frame_gray,previousPoints, None,**lk_params)
    
    good_new = nextPoints[status == 1]
    good_previous = previousPoints[status ==1]
    
    for i,(new,previous) in enumerate(zip(good_new, good_previous)):
        
        x_new, y_new = new.ravel()
        x_previous, y_previous = previous.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_previous,y_previous),(0,255,0),3)
        
        frame = cv2.circle(frame,(x_new, y_new),8 ,(0,0,255),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    
    previous_gray = frame_gray.copy()
    previousPoints = good_new.reshape(-1,1,2)
               
cv2.destroyAllWindows()
cap.release()
    
               

# Dense Optical Flow in OpenCV

calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow

This function computes a dense optical flow using the Gunnar Farneback's algorithm.

Here are the parameters for the function and what they represent:

* prev first 8-bit single-channel input image.
* next second input image of the same size and the same type as prev.
* flow computed flow image that has the same size as prev and type CV_32FC2.
* pyr_scale parameter, specifying the image scale (\<1) to build pyramids for each image
    * pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.
    
* levels number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.
* winsize averaging window size
    * larger values increase the algorithm robustness to image
* noise and give more chances for fast motion detection, but yield more blurred motion field.
* iterations number of iterations the algorithm does at each pyramid level.
* poly_n size of the pixel neighborhood used to find polynomial expansion in each pixel
    * larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.
* poly_sigma standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.

In [5]:
import cv2 
import numpy as np

# Capture the frame
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

# Get gray scale image of first frame and make a mask in HSV color
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
# Here we change the saturation value to 255.
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    # Check out the markdown text above for a break down of these paramters, most of these are just suggested defaults
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    
    # Color the channels based on the angle of travel
    # Pay close attention to your video, the path of the direction of flow will determine color!
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # Convert back to BGR to show with imshow from cv
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Set the Previous image as the next image for the loop
    prvsImg = nextImg

    
cap.release()
cv2.destroyAllWindows()

On moving from right to left, its blue and red for the oopposite and thus different colors for different directions. 

# MeanShift Tracking

In [6]:
import numpy as np
import cv2 

# Capture a video stream
cap = cv2.VideoCapture(0)

# take first frame of the video
ret,frame = cap.read()


# Set Up the Initial Tracking Window


# We will first detect the face and set that as our starting box.
face_cascade = cv2.CascadeClassifier('/home/soniya/Documents/work/Computer-Vision-with-Python/DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame) 

# Convert this list of a single array to a tuple of (x,y,w,h)
(face_x,face_y,w,h) = tuple(face_rects[0]) 
track_window = (face_x,face_y,w,h)
# set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]


# Use the HSV Color Mapping
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Find histogram to backproject the target on each frame for calculation of meanshit
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalize the histogram array values given a min of 0 and max of 255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:
        
        # Grab the Frame in HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Calculate the Back Projection based off the roi_hist created earlier
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # Apply meanshift to get the new coordinates of the rectangle
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        # Draw the new rectangle on the image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),5)
        
        cv2.imshow('img2',img2)
        
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break
        
cv2.destroyAllWindows()
cap.release()

# CamShift Tracking

In [11]:
import numpy as np
import cv2 

# Capture a video stream
cap = cv2.VideoCapture(0)

# take first frame of the video
ret,frame = cap.read()


# Set Up the Initial Tracking Window


# We will first detect the face and set that as our starting box.
face_cascade = cv2.CascadeClassifier('/home/soniya/Documents/work/Computer-Vision-with-Python/DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame) 

# Convert this list of a single array to a tuple of (x,y,w,h)
(face_x,face_y,w,h) = tuple(face_rects[0]) 
track_window = (face_x,face_y,w,h)
# set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]


# Use the HSV Color Mapping
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Find histogram to backproject the target on each frame for calculation of meanshit
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalize the histogram array values given a min of 0 and max of 255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:
        
        # Grab the Frame in HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Calculate the Back Projection based off the roi_hist created earlier
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        #########################################################
        #########################################################
        ########## CAM SHIFT ####################################
        ########################################################
        #######################################################
        
        # Apply Camshift to get the new coordinates of the rectangle
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, (0,0,255),5)
        cv2.imshow('img2',img2)
        
        ########################################################
        #######################################################
        ########################################################
        #######################################################
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break
        
cv2.destroyAllWindows()
cap.release()